In [6]:
import glob,os
import pandas as pd
import numpy as np
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import warnings
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.dates as mdates
from datetime import datetime, timedelta

warnings.filterwarnings("ignore")

In [7]:
oripath = '../Results/soundsepration'
dp = pd.read_csv('../Results/soundsepration/experiment_period.txt')

In [8]:
def figure_base_setting(ax):
    ax[0].set_xlim(datetime(2021, 4, 1) ,datetime(2022, 1, 10))
    ax[1].set_xlim(datetime(2022, 4, 1) ,datetime(2023, 1, 10))
    ax[2].set_xlim(datetime(2023, 4, 1) ,datetime(2024, 1, 10))

    ax[0].grid(axis='y', color='lightgrey')
    ax[1].grid(axis='y', color='lightgrey')
    ax[2].grid(axis='y', color='lightgrey')

    ax[0].yaxis.set_minor_locator(MultipleLocator(0.25))
    ax[0].yaxis.set_major_locator(MultipleLocator(0.5))
    
    ax[0].yaxis.set_ticks_position('both')
    ax[1].yaxis.set_ticks_position('both')
    ax[2].yaxis.set_ticks_position('both')

    ax[0].xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax[1].xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax[2].xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    
    ax[1].xaxis.set_major_formatter(mdates.DateFormatter("%b"))
    ax[0].xaxis.set_major_formatter(mdates.DateFormatter("%b"))
    ax[2].xaxis.set_major_formatter(mdates.DateFormatter("%b"))


In [ ]:
plt.rcParams["font.family"] = "DejaVu Sans"
fig, ax = plt.subplots(1,3,figsize=(15,4), sharey=True)
plt.subplots_adjust(wspace = 0.09) 

            
# for idx in range(1, 5):
#     dp_ = dp[dp['region']=='Keelung'][dp['year']==int('2023')][dp['index']==idx]
#     begin = dp_['begin'].values[0]; end = dp_['end'].values[0]
#     begintime = datetime.strptime(begin, "%Y-%m-%dT%H:%M:%S")
#     endtime = datetime.strptime(end, "%Y-%m-%dT%H:%M:%S") 
#     ax[1].hlines(xmin=begintime, xmax=endtime, y=-0.05, lw=7, color='orangered', alpha=0.5)   
    
for path in glob.glob(f'{oripath}/*-5min/cluster_entire_3years/*log'):
    df = pd.read_csv(path)
    color = 'orangered' if 'Keelung' in path else 'dodgerblue'
    region = 'Keelung' if 'Keelung' in path else 'Taoyuan'
    dx = -0.15 if 'Keelung' in path else 0.15

    for year in [2021,2022,2023]:
        if int(year) == 2021:
            pidx = 0 
        elif int(year) == 2022:
            pidx = 1
        elif int(year) == 2023:
            pidx = 2  
        ax[pidx].set_title(year, fontsize=15, fontweight='bold')
        
        df_year = df[df['year']==year]
        for idx in range(len(df_year)):
            idxx = idx+1
            pct5 = df_year['pct5'].values[idx]
            Q1 = df_year['Q1'].values[idx]
            Q2 = df_year['Q2'].values[idx]
            Q3 = df_year['Q3'].values[idx]
            pct95 = df_year['pct95'].values[idx]
            dp_ = dp[dp['region']==region][dp['year']==int(year)][dp['index']==idxx]
            begin = dp_['begin'].values[0]; end = dp_['end'].values[0]
            begintime = datetime.strptime(begin, "%Y-%m-%dT%H:%M:%S")
            endtime = datetime.strptime(end, "%Y-%m-%dT%H:%M:%S")
            dt = endtime - begintime
            if year == 2023:
                if idxx == 3 or idxx==4 :
                    ddt = 0 if region == 'Taoyuan' else 0
                    align = 'right' if region == 'Taoyuan' else 'left'
                else:
                    ddt = -4 if region == 'Keelung' else 4
                    align = 'right' if region == 'Keelung' else 'left'
            else:
                ddt = -4 if region == 'Keelung' else 4
                align = 'right' if region == 'Keelung' else 'left'
            plottime = begintime + timedelta(days=dt.days/2+ddt)      

            ax[pidx].hlines(xmin=begintime, xmax=endtime, y=-0.05, lw=7, color=color, alpha=0.5)
            ax[pidx].plot([plottime, plottime], [pct5, pct95], lw=2, color=color)
            ax[pidx].plot([plottime, plottime], [Q1, Q3], lw=4, color=color, label = region)
            ax[pidx].plot([plottime], [Q2], marker = 'o', markersize = 6, color='white', mec='k', mew=1.5)# ,edgecolors='k', lw=2
            if year == 2023:   
                if idxx == 3 or idxx==4:
                    if region == 'Taoyuan':
                        ax[pidx].text(plottime + timedelta(days=-4), Q2, float("{:.2f}".format(Q2,2)), color=color, alpha=0.7,
                                  ha=align, va ='bottom', fontsize=7, fontweight='heavy')
                    elif region == 'Keelung':
                        ax[pidx].text(plottime + timedelta(days=4), Q2, float("{:.2f}".format(Q2,2)), color=color, alpha=0.7,
                                  ha=align, va ='bottom', fontsize=7, fontweight='heavy')
                else:
                    ax[pidx].text(plottime + timedelta(days=ddt), Q2, float("{:.2f}".format(Q2,2)), color=color, alpha=0.7,
                              ha=align, va ='bottom', fontsize=7, fontweight='heavy')                      
            else:
                ax[pidx].text(plottime + timedelta(days=ddt), Q2, float("{:.2f}".format(Q2,2)), color=color, alpha=0.7,
                              ha=align, va ='bottom', fontsize=7, fontweight='heavy')  

figure_base_setting(ax)
    
ax[0].set_ylim(-0.1, 2.5)
ax[0].set_ylabel('Shannon Diversity Index\n', fontsize=12)

handles, labels = ax[2].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), fontsize=10)


plt.savefig('../Results/soundsepration/Finalresults/Index_3year_cluster3year_v5.pdf', dpi=200, transparent=True, )
plt.savefig('../Results/soundsepration/Finalresults/Index_3year_cluster3year_v5.png', dpi=200, bbox_inches='tight', transparent=True, )

NameError: name 'plt' is not defined